# Erstellen von KI-Lösungen mit Azure Machine Learning

## 1 Einführung in Azure Machine Learning

Lernziele:
- Bereitstellen eines Azure Machine Learning-Arbeitsbereichs
- Verwenden von Tools und Benutzeroberflächen für die Arbeit mit Azure Machine Learning
- Ausführen von auf Code basierenden Experimenten in einem Azure Machine Learning-Arbeitsbereich

### Arbeitsbereiche für Machine Learning-Ressourcen

- Computeziele für die Entwicklung, das Training und die Bereitstellung
- Daten für Experimente und Modelltraining
- Notebooks mit freigegebenem Code und Dokumentation
- Experimente, einschließlich Ausführungsverlauf mit protokollierten Metriken und Ausgaben
- Pipelines, die orchestrierte Prozesse mit mehreren Schritten definieren
- Modelle, die Sie trainiert haben

![image](Bilder/01-02-workspace.jpg)

Folgende Azure-Ressourcen werden gemeinsam mit einem Arbeitsbereich erstellt:

- Ein Speicherkonto, das zum Speichern von Dateien, die vom Arbeitsbereich genutzt werden, und der Daten für Experimente und Modelltraining verwendet wird
- Eine Application Insights-Instanz, die zum Überwachen der Vorhersagedienste im Arbeitsbereich verwendet wird
- Eine Azure Key Vault-Instanz, die zum Verwalten von Geheimnissen wie Authentifizierungsschlüsseln und Anmeldeinformationen für den Arbeitsbereich verwendet wird
- VMs (virtuelle Computer) und die zugehörigen virtuellen Hardwareressourcen, die zur Bereitstellung für die Notebookentwicklung im Arbeitsbereich verwendet werden.
- Eine Containerregistrierung, die zum Verwalten von Containern für bereitgestellte Modelle verwendet wird

In [ ]:
# Erstellen eines Arbeitsbereichs über die Python SDK und CLI-Erweiterung
from azureml.core import Workspace
    
    ws = Workspace.create(name='aml-workspace', 
                      subscription_id='123456-abc-123...',
                      resource_group='aml-resources',
                      create_resource_group=True,
                      location='eastus',
                      sku='enterprise'
                     )

az ml workspace create -w 'aml-workspace' -g 'aml-resources'

### Tools und Benutzeroberflächen von Azure Machine Learning

In [ ]:
# Herstellen einer Verbindung mit einem Arbeitsbereich über config.json

#json
{
    "subscription_id": "1234567-abcde-890-fgh...",
    "resource_group": "aml-resources",
    "workspace_name": "aml-workspace"
}

#python
from azureml.core import Workspace

ws = Workspace.from_config()

#oder

ws = Workspace.get(name='aml-workspace',
                   subscription_id='1234567-abcde-890-fgh...',
                   resource_group='aml-resources') 

### Azure Machine Learning-Experimente

In [ ]:
# Ausführungskontext
from azureml.core import Experiment

# create an experiment variable
experiment = Experiment(workspace = ws, name = "my-experiment")

# start the experiment
run = experiment.start_logging()

# Log the row count examplelog
run.log('observations', row_count)

# end the experiment
run.complete()

In [ ]:
# Augabe des Logs

# als print
from azureml.widgets import RunDetails
RunDetails(run).show()

# als json
import json
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

# direkt in eigenes file in Output-Ordner
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')


In [ ]:
## 2 Trainieren eines Machine Learning Modells mit Azure ML

Lernziele:
- Verwenden eines Estimators zum Ausführen eines Modelltrainingsskripts als Azure Machine Learning-Experiment
- Erstellen von wiederverwendbaren, parametrisierten Trainingsskripts
- Registrieren von Modellen, einschließlich Metadaten wie Leistungsmetriken

für

- Erstellen Sie wiederverwendbare Trainingsskripts für mehrere Computeumgebungen.
- Protokollieren Sie Modellleistungsmetriken während der Trainings-und Bewertungsprozesse.
- Verfolgen Sie Modellversionen.

### Run a training script

Experiment notmalerweise innerhalb eines separaten Skripts geschrieben, dass dann nochmal über **RunConfiguration** und **ScriptRunConfig** ausgeführt wird.

In [ ]:
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig

# create a new RunConfig object
experiment_run_config = RunConfiguration()

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder, 
                      script='experiment.py',
                      run_config=experiment_run_config) 

# submit the experiment
experiment = Experiment(workspace = ws, name = 'my-experiment')
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)

In [ ]:
# oder mit festlegen einer eigenen environment
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# Create a script config
script_config = ScriptRunConfig(source_directory='training_folder',
                                script='training.py',
                                environment=sklearn_env) 

# 
experiment = Experiment(workspace=ws, name='training-experiment')
run = experiment.submit(config=script_config)
run.wait_for_completion()

In [ ]:
### Verwenden von Skriptparametern

In [ ]:
## im Trainingsscript einfügen. reg kann danach als normale Variable weiterverwendet werden

import argparse
# Set regularization hyperparameter
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg


## in der Ausfürhung vor dem Submitten noch einen estimater erstellen und in die config packen
## Nennung von reg als script_params
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment

# Create an estimator
estimator = SKLearn(source_directory='experiment_folder',
                    entry_script='training_script.py',
                    script_params = {'--reg_rate': 0.1},
                    compute_target='local'
                    )

# Submit the experiment
experiment = Experiment(workspace = ws, name = 'training_experiment')
run = experiment.submit(config=estimator)

### Modell wird registriert

Skripte speichern die Trainierten modelle standarsmäßig im Output-Ordner. Von dort aus können sie auch heruntergeladen werden.

Mithilfe der Modellregistrierung können Sie mehrere Versionen eines Modells nachverfolgen und Modelle für Rückschlüsse (Vorhersagen von Bezeichnungswerten aus neuen Daten) abrufen. Wenn Sie ein Modell registrieren, können Sie einen Namen, eine Beschreibung, Tags, ein Framework (wie Scikit-Learn oder PyTorch), Frameworkversion, benutzerdefinierte Eigenschaften und andere nützliche Metadaten angeben. Wenn Sie ein Modell mit dem gleichen Namen wie ein vorhandenes Modell registrieren, wird automatisch eine neue Version des Modells erstellt, beginnend mit 1 und aufsteigend in Einheiten von 1.

In [ ]:
# Download a named file
run.download_file(name='outputs/model.pkl', output_file_path='model.pkl')

# Registrierung über Register Methode des Modell-Objekts
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='classification_model',
                       model_path='model.pkl', # local path
                       description='A classification model',
                       tags={'dept': 'sales'},
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.20.3')

# über Run-Methode
run.register_model( model_name='classification_model',
                    model_path='outputs/model.pkl', # run outputs path
                    description='A classification model',
                    tags={'dept': 'sales'},
                    model_framework=Model.Framework.SCIKITLEARN,
                    model_framework_version='0.20.3')

In [ ]:
## 3 Work with Data in Azure ML

In [ ]:
## 4 Work with Compute in Azure ML

In [ ]:
## 5 Orchestrate machine learning with pipelines

In [ ]:
## 6 Bereitstellen von Machine Learning Modellen mit Azure ML

In [ ]:
## 7 Delpoy batch inference pipelines with Azure ML

In [ ]:
## 8 Tune hyperparameters with Azure ML

In [ ]:
## 9 Automatisieren der ML-Modellauswahl mit Azure ML

In [ ]:
## 10 Explore differential privacy

In [ ]:
## 11 Explain machine leraning models with Azure ML

In [ ]:
## 12 Detect and mitigate unfaireness in models with Azure ML

In [ ]:
## 13 Monitor models with Azure ML

In [ ]:
## 14 Monitor data drift with Azure ML